In [1]:
import pandas as pd

wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(wikipedia_link)[0]
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df = df.iloc[1:]

df = df[df['Borough']!='Not assigned']

index = df.loc[df['Neighborhood']=='Not assigned'].index.values
df.loc[index, 'Neighborhood'] = df.loc[index, 'Borough']

series = df.groupby(['PostalCode']).apply(lambda x: ', '.join(x['Neighborhood']))
df2 = series.to_frame().reset_index()
df2.columns = ['PostalCode', 'Neighborhood']
df = pd.merge(df[['PostalCode','Borough']], df2, on='PostalCode', how='right')
df = df.drop_duplicates(['PostalCode'])  

df = df.reset_index(drop=True)
df.shape

(103, 3)

#Rows = 103
#Columns = 3

In [2]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [3]:
!pip install --user geocoder

    100% |████████████████████████████████| 102kB 7.2MB/s ta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

In [4]:
import geocoder # import geocoder

latitude = []
longitude = []

for borough in df['Borough']:
    g = geocoder.osm('{}, Toronto, Ontario'.format(borough))
    latitude.append(g.osm['y'])
    longitude.append(g.osm['x'])
    
df3 = pd.DataFrame({'PostalCode': df['PostalCode'], 'Latitude': latitude}) 
df4 = pd.DataFrame({'PostalCode': df['PostalCode'], 'Longitude': longitude})
df_new = pd.merge(df, df3, on = 'PostalCode')
df_new = pd.merge(df_new, df4, on = 'PostalCode')
df_new.head(10)

    # initialize your variable to None
# lat_lng_coords = None
    # loop until you get the coordinates
# while(lat_lng_coords is None):
    # g = geocoder.google('{}, Toronto, Ontario'.format(pc))
    # lat_lng_coords = g.latlng

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.770817,-79.413300
1,M4A,North York,Victoria Village,43.770817,-79.413300
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655115,-79.380219
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.770817,-79.413300
4,M7A,Queen's Park,Queen's Park,43.659980,-79.390369
5,M9A,Etobicoke,Islington Avenue,43.671459,-79.552492
6,M1B,Scarborough,"Rouge, Malvern",43.773077,-79.257774
7,M3B,North York,Don Mills North,43.770817,-79.413300
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.691339,-79.327821
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.655115,-79.380219


In [5]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [6]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.66 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.02 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.98 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.15 MB/s
vincent-0.4.4- 100% |###################

In [8]:
# parameter for foursquare-api

# CLIENT_ID =
# CLIENT_SECRET =
# VERSION =
# LIMIT =


- get information about the venue categories within a certain radius of the centre of the borough from foursquare via foursquare-api
- this information is used for the k-mean algorithm
- the number of each category in a borough corresponds to a feature for the algorithm

In [10]:
radius = 250
n = len(latitude)
filtered_columns = ['PostalCode', 'Borough', 'venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_new2 = pd.DataFrame([], columns = filtered_columns)
for i in range(0, n):
    lati = latitude[i]
    long = longitude[i]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lati, long, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    
    try:
        items = results['response']['groups'][0]['items']
        dataframe = json_normalize(items)
        m = len(dataframe)
        dataframe['PostalCode'] = pd.DataFrame({'PostalCode': [df_new.loc[i, 'PostalCode']] * m})
        dataframe['Borough'] = pd.DataFrame({'Borough': [df_new.loc[i, 'Borough']] * m})
        dataframe['venue.categories'] = dataframe['venue.categories'].apply(lambda x: x[0]['name'])
        dataframe_fil = dataframe.loc[:, filtered_columns]
        df_new2 = df_new2.append(dataframe_fil)
    except KeyError:
        pass

In [11]:
df_new2.head(10)

,PostalCode,Borough,venue.id,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,M3A,North York,557767bc498ea4a20c8043f6,Aroma Espresso Bar,Café,43.769449,-79.413081
1,M3A,North York,4ae257cff964a520758d21e3,Loblaws,Grocery Store,43.768648,-79.412597
2,M3A,North York,53c7201c498ef6785edb6856,Dakgogi,Korean Restaurant,43.773010,-79.413875
3,M3A,North York,4ad4c062f964a520e2f720e3,Cineplex Cinemas Empress Walk,Movie Theater,43.768625,-79.412613
4,M3A,North York,4b315d90f964a520840525e3,Ajisen Ramen 味千ラーメン,Ramen Restaurant,43.771444,-79.413139
5,M3A,North York,4b590484f964a520b27828e3,St. Louis Bar & Grill,Wings Joint,43.773040,-79.413771
6,M3A,North York,5ab5accd6bd36b087487be93,Aburi Room,Sushi Restaurant,43.769197,-79.414039
7,M3A,North York,544ad69a498eeb86b7a4a8d0,KINTON RAMEN,Ramen Restaurant,43.769684,-79.413049
8,M3A,North York,5077025ce4b05aefca37d9f9,Wako Sushi + Bar,Japanese Restaurant,43.770806,-79.413138
9,M3A,North York,5a0c5ec8fdb9a7258ac7f16d,PetSmart,Pet Store,43.769139,-79.412522


In [12]:
countCategoriesPerBorough = df_new2.groupby(['Borough','venue.categories']).count()['PostalCode'].reset_index()

list_borough = countCategoriesPerBorough['Borough'].unique()
y = pd.DataFrame(list_borough, columns = ['Borough'])

list_venueCategories = countCategoriesPerBorough['venue.categories'].unique()
df_allVenueCategories = pd.DataFrame(list_venueCategories, columns = ['venue.categories'])

X = pd.DataFrame([])

for borough in list_borough:
    df_CategoriesPerBorough = countCategoriesPerBorough[countCategoriesPerBorough['Borough'] == borough][['venue.categories','PostalCode']]
    df_CategoriesPerBorough = df_CategoriesPerBorough.append(pd.DataFrame({'venue.categories': list(set(df_allVenueCategories['venue.categories'])-set(df_CategoriesPerBorough['venue.categories'])), 'PostalCode': [0] * len(list(set(df_allVenueCategories['venue.categories'])-set(df_CategoriesPerBorough['venue.categories'])))})).reset_index(drop=True)
    df_mapping = pd.merge(df_allVenueCategories, df_CategoriesPerBorough, on = 'venue.categories', how = 'left')
    X = X.append(df_mapping.T.iloc[1])
    
X = X.reset_index(drop = True)
X.columns = list_venueCategories
X

,American Restaurant,Bakery,Beer Bar,Café,Cocktail Bar,Coffee Shop,Fried Chicken Joint,Hotel,Hotel Bar,Irish Pub,...,Bus Station,Chocolate Shop,Discount Store,Light Rail Station,Sporting Goods Shop,Video Game Store,Fast Food Restaurant,Lawyer,Museum,Skating Rink
0,9.0,18.0,9.0,9.0,9.0,72.0,9.0,18.0,9.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,18.0,0.0,18.0,18.0,0.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,24.0,0.0,24.0,0.0,24.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,17.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,...,17.0,17.0,17.0,17.0,17.0,17.0,0.0,0.0,0.0,0.0
8,6.0,12.0,6.0,6.0,6.0,48.0,6.0,12.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0


In [29]:
k_means = KMeans(init="k-means++", n_clusters=4, n_init=12)
k_means.fit(X)

k_means_labels = k_means.labels_
k_means_cluster_centers = k_means.cluster_centers_

yWithClusterNumber = pd.concat([y.reset_index(drop=True), pd.DataFrame(k_means_labels.tolist(), columns = ["Cluster number"])], axis=1)
yWithClusterNumber = pd.merge(df_new[['Borough', 'Latitude', 'Longitude']], yWithClusterNumber, on='Borough', how='right').groupby(['Borough']).mean().reset_index()

yWithClusterNumber

,Borough,Latitude,Longitude,Cluster number
0,Central Toronto,43.644903,-79.381836,0
1,Downtown Toronto,43.655115,-79.380219,2
2,East York,43.691339,-79.327821,1
3,Etobicoke,43.671459,-79.552492,1
4,Mississauga,43.666403,-79.588145,1
5,North York,43.770818,-79.413300,3
6,Queen's Park,43.659980,-79.390369,1
7,Scarborough,43.773077,-79.257774,1
8,West Toronto,43.644903,-79.381836,0
9,York,43.689619,-79.479188,1


In [ ]:
address = 'Toronto, Ontario'

g = geocoder.osm(address)
latitude = g.osm['y']
longitude = g.osm['x']

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
n = len(yWithClusterNumber)

for i in range(n):
    borough = yWithClusterNumber.iloc[i]
    
    if borough["Cluster number"] == 0:
        color = 'red'
    elif borough["Cluster number"] == 1:
        color = 'blue'
    elif borough["Cluster number"] == 2:
        color = 'yellow'
    elif borough["Cluster number"] == 3:
        color = 'green'
    
    label = '{}'.format(borough["Borough"])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [borough["Latitude"], borough["Longitude"]],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
# add Ecco as a red circle mark
#folium.features.CircleMarker(
 #   [latitude, longitude],
  #  radius=10,
   # popup='Ecco',
 #   fill=True,
  #  color='red',
   # fill_color='red',
 #   fill_opacity=0.6
  #  ).add_to(map_toronto)

        
# display map
display(map_toronto)
map_toronto.save("toronto_map.html")